In [ ]:
from openai import OpenAI
from pathlib import Path
import os
from dotenv import load_dotenv
import time
import re

load_dotenv()

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=os.environ['OPENROUTER_API_KEY']
)

output = Path('output')


In [ ]:

start_time = time.time()
completion = client.chat.completions.create(
  #model="deepseek/deepseek-chat-v3-0324:free",
  #model='google/gemini-2.5-pro-exp-03-25',
  #model='deepseek/deepseek-r1:free',
  #model='google/gemini-2.0-flash-exp:free',
  #model='meta-llama/llama-4-maverick:free',
  #model='deepseek/deepseek-chat:free',
  #model='microsoft/mai-ds-r1:free',
  #model='qwen/qwen3-235b-a22b:free',
  #model='meta-llama/llama-4-scout:free',
  #model='tngtech/deepseek-r1t-chimera:free',
  #model='nvidia/llama-3.1-nemotron-ultra-253b-v1:free',
  #model='google/gemma-3-27b-it:free',
  model='mistralai/mistral-small-3.1-24b-instruct:free',
  messages=[
    {
      "role": "user",
      "content": "Generate a plausible log file, as would be emitted from some application. It should contain both uninteresting and interesting lines. Reply ONLY with the log lines. No explanations, markdown quotes or any other form of framing."
    }
  ]
)
end_time = time.time()

content = completion.choices[0].message.content
print(content)
output.mkdir(exist_ok=True, parents=True)
for i in range(0, 1000000):
    outpath = output / f'{i}.log'
    if outpath.exists():
        continue
    break
else:
    assert False
with outpath.open('w') as f:
    f.write(content)

timings_path = Path('timings.txt')
with timings_path.open('a') as f:
    print(f'{outpath}\t{completion.model}\t{end_time - start_time}', file=f)


In [ ]:
format_check = []
for p in output.glob('*.log'):
    logfile = p.read_text()
    def fun(msg):
      match msg:
          case 'yes':
            return True
          case 'no':
            return False
          case _:
            raise Exception(f'bad reply from model: {msg}')
    for retry in range(100000):
      try:
        completion = client.chat.completions.create(
          model='meta-llama/llama-4-maverick:free',
          messages=[
            {
              "role": "user",
              "content": f"Does the following text look like a raw log file? It mustn't contain any additional framing, only the actual log lines. Answer ONLY with yes or no:\n{logfile}"
            }
          ]
        )
        good = fun(completion.choices[0].message.content)
      except Exception as e:
        if retry < 10:
          print(f'bad reply from model: {e}. retrying..')
        else:
          raise
    format_check.append((p, good))
    print(f'{p}: {good}')
          
